# LMNA

In [1]:
import genophenocorr
print(f"Using genophenocorr version {genophenocorr.__version__}")

/Users/robin/PycharmProjects/genophenocorr/gpc_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Using genophenocorr version 0.1.1dev


In [18]:
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo()

In [19]:
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder

fpath_phenopackets = 'input'
cohort_creator = configure_caching_cohort_creator(hpo)
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)

Patients Created: 100%|██████████| 127/127 [00:00<00:00, 622.02it/s]
Validated under none policy
Showing errors and warnings
127 phenopacket(s) found at `input`
  patient #0
    phenotype-features
     warnings:
     ·No diseases found.
  patient #1
    phenotype-features
     warnings:
     ·No diseases found.
  patient #2
    phenotype-features
     warnings:
     ·No diseases found.
  patient #3
    phenotype-features
     warnings:
     ·No diseases found.
  patient #4
    phenotype-features
     warnings:
     ·No diseases found.
  patient #5
    phenotype-features
     warnings:
     ·No diseases found.
  patient #6
    phenotype-features
     warnings:
     ·No diseases found.
  patient #7
    phenotype-features
     warnings:
     ·No diseases found.
  patient #8
    phenotype-features
     warnings:
     ·No diseases found.
  patient #9
    phenotype-features
     warnings:
     ·No diseases found.
  patient #10
    phenotype-features
     warnings:
     ·No diseases found.
  

## Transcript
Get it by looking for gene symbol in https://www.ncbi.nlm.nih.gov/clinvar/ look for pathogenic
zcopy it and go to https://variantvalidator.org/service/validate/, then scroll down to copy GRCH38 string, copy this and put into same website

looking for MANE select plus clinical (most important transcript for clinic) if doesnt exist, use clinvar result

In [20]:
tx_id = 'NM_170707.4'
protein_id = 'NP_001027558.1'

from genophenocorr.view._draw_variants import VariantsVisualizer
from genophenocorr.model.genome import GRCh38
from genophenocorr.preprocessing import VVTranscriptCoordinateService

txc_service = VVTranscriptCoordinateService(genome_build=GRCh38)
tx_coordinates = txc_service.fetch(tx_id)

Response.transcripts has 6!=1 items. Choosing the first


In [21]:
from genophenocorr.preprocessing import UniprotProteinMetadataService

pms = UniprotProteinMetadataService()

protein_metas = pms.annotate(protein_id)

assert len(protein_metas) == 1
protein_meta = protein_metas[0]

In [17]:
cohort

In [23]:
from genophenocorr.view._draw_variants import VariantsVisualizer
viz = VariantsVisualizer()
viz.draw_fig(tx_coordinates, protein_meta, cohort)

AttributeError: 'NoneType' object has no attribute 'start'

## Explore Cohort

In [5]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewer

viewer = CohortViewer(hpo)

In [6]:
display(HTML(viewer.cohort_summary_table(cohort)))

Item,Description
Total Individuals,127
Total Unique HPO Terms,214
Total Unique Variants,127


In [7]:
display(HTML(viewer.hpo_term_counts_table(cohort))) ## Add Labels to output

HPO Term,Count
Ankle contracture (HP:0034677),39
Elbow contracture (HP:0034391),37
Spinal rigidity (HP:0003306),37
Stiff neck (HP:0025258),36
Proximal muscle weakness in upper limbs (HP:0008997),35
Dilated cardiomyopathy (HP:0001644),27
Atrial fibrillation (HP:0005110),25
Arrhythmia (HP:0011675),23
Distal lower limb muscle weakness (HP:0009053),21
Lipodystrophy (HP:0009125),21


In [8]:
display(HTML(viewer.variants_table(cohort, tx_id))) 

Variant,Effect,Count,Key
c.1824C>T,SYNONYMOUS_VARIANT,15,1_156138613_156138613_C_T
c.585C>A,MISSENSE_VARIANT,11,1_156134474_156134474_C_A
c.1698+13C>A,INTRON_VARIANT,8,1_156137756_156137756_C_A
c.608A>G,MISSENSE_VARIANT,8,1_156134497_156134497_A_G
c.1580G>C,MISSENSE_VARIANT,8,1_156137204_156137204_G_C
c.178C>G,MISSENSE_VARIANT,7,1_156115096_156115096_C_G
c.1444C>T,MISSENSE_VARIANT,7,1_156136984_156136984_C_T
c.1072G>A,MISSENSE_VARIANT,6,1_156136036_156136036_G_A
c.1357C>T,MISSENSE_VARIANT,6,1_156136413_156136413_C_T
c.16C>T,STOP_GAINED,5,1_156114934_156114934_C_T


## Configure the analysis

In [ ]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from genophenocorr.analysis.predicate import BooleanPredicate, GroupingPredicate

analysis_config = CohortAnalysisConfiguration.builder() \
    .missing_implies_excluded(True) \
    .pval_correction('fdr_bh') \
    .min_perc_patients_w_hpo(0.1) \
    .build()
analysis = configure_cohort_analysis(cohort, hpo, analysis_config)

# did not work because we didn#'t get the right transcript
# frameshift = analysis.compare_by_variant_effect(VariantEffect.FRAMESHIFT_VARIANT, tx_id)
# frameshift.summarize(hpo, BooleanPredicate.YES)
var_single = analysis.compare_by_variant_key('1_156138613_156138613_C_T')
var_single.summarize(hpo, BooleanPredicate.YES)